# DSCI100 Project: Predicting Playtime Range from Age and Experience

#### Group 42
#### Members: Linda Zhu, Eunelsy Trillanes, Lavender Sun, and Kelly Ye

*Based on a player's age and experience, can we predict the range of hours — zero, low, medium, high, or extreme — that they will contribute?*